In [17]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader, UnstructuredWordDocumentLoader,DirectoryLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings,HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.schema import Document

import requests
from bs4 import BeautifulSoup
import os

c:\Users\Dell\anaconda3\envs\chat\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
def select_loader(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.pdf':
        return PyPDFLoader(file_path)
    elif ext == '.csv':
        return CSVLoader(file_path)
    elif ext == '.docx' or ext == '.doc':
        return Docx2txtLoader(file_path)
    else:
        return None

def load_all_files(directory_path):
    combined_text = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            loader = select_loader(file_path)
            if loader:
                documents = loader.load()
                combined_text += documents
    
    return combined_text

def crawl_web(urls):
    combined_text = []
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            # Loại bỏ các phần không cần thiết như script, style
            for script in soup(["script", "style"]):
                script.decompose()
            text = soup.get_text(separator='\n')
            combined_text.append(text)
        except Exception as e:
            print(f"Không thể crawl {url}: {e}")
    return combined_text


In [13]:
directory_path = "data"
urls_to_crawl = [
    "https://thptbuithixuan.hcm.edu.vn/homegd3",
]
web_documents = crawl_web(urls_to_crawl)
documents = load_all_files(directory_path)
processed_web_documents = [Document(page_content=doc, metadata={}) for doc in web_documents]


combined_documents = processed_web_documents + documents


In [14]:
combined_documents

[Document(metadata={}, page_content='\n\n\n\n\n\n\r\n\tTRƯỜNG THPT BÙI THỊ XUÂN\r\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nKhối THPT\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n Ngành Giáo Dục Và Đào Tạo Thành Phố Hồ Chí Minh\n TRƯỜNG THPT BÙI THỊ XUÂN\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGiới thiệu\nCông khai thông tin\nBiểu mẫu\nKế hoạch công tác\nTuyển sinh\nTin tức - sự kiện\nCải cách hành chính\nChính trị - tư tưởng\nY tế học đường\nHoạt động chuyên môn\nGÓC HỌC TẬP TRỰC TUYẾN\nTHÔNG BÁO\nHƯỚNG DẪN\nTHỜI KHÓA BIỂU\nliên hệ \n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGiới thiệu\n \n&nbspCông khai thông tin\n \n&nbspBiểu mẫu\nThi - kiểm tra\n \n&nbspKiểm tra học kỳ 1\n \n\xa0\xa0\n&nbspMa trận\n \n\xa0\xa0\n\xa0Đề - Đáp án\n \n\xa0\xa0\n&nbspVăn\n \n\xa0\xa0\n&nbspToán\n \n\xa0\xa0\n&nbspLý\n \n\xa0\xa0\n&nbspHóa\n \n\xa0\xa0\n&nbspTiếng Anh\n \n\xa0\xa0\n&nbspSinh\n \n\xa0\xa0\n&n

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = text_splitter.split_documents(combined_documents)

# Embeding
# embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

db = FAISS.from_documents(chunks, embedding_model)

c:\Users\Dell\anaconda3\envs\chat\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Dell\anaconda3\envs\chat\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer M

In [10]:
model_file = 'vinallama-7b-chat_q5_0.gguf'
llm = CTransformers(
        model=model_file,
        model_type="llama",
        max_new_tokens=1024,
        temperature=0.01
    )

c:\Users\Dell\anaconda3\envs\chat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
template = """<|im_start|>system\nSử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời\n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""
prompt = PromptTemplate(template = template, input_variables=["context", "question"])
llm_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff",
        retriever = db.as_retriever(search_kwargs = {"k":3}, max_tokens_limit=1024),
        return_source_documents = False,
        chain_type_kwargs= {'prompt': prompt}

    )

In [38]:
question = "bạn hãy giới thiệu về bản thân"
response = llm_chain.invoke({"query": question})
print(response)

Number of tokens (627) exceeded maximum context length (512).
Number of tokens (628) exceeded maximum context length (512).
Number of tokens (629) exceeded maximum context length (512).
Number of tokens (630) exceeded maximum context length (512).
Number of tokens (631) exceeded maximum context length (512).
Number of tokens (632) exceeded maximum context length (512).
Number of tokens (633) exceeded maximum context length (512).
Number of tokens (634) exceeded maximum context length (512).
Number of tokens (635) exceeded maximum context length (512).
Number of tokens (636) exceeded maximum context length (512).
Number of tokens (637) exceeded maximum context length (512).
Number of tokens (638) exceeded maximum context length (512).
Number of tokens (639) exceeded maximum context length (512).
Number of tokens (640) exceeded maximum context length (512).
Number of tokens (641) exceeded maximum context length (512).
Number of tokens (642) exceeded maximum context length (512).
Number o

{'query': 'bạn hãy giới thiệu về bản thân', 'result': '> >1>|>tant|>  và các chi tiết;  \n-g.\n-s  và thông tin, địa điểm;  \n-thand  tance,  tance.\n-\n-'}


In [39]:
print(response['result'])

> >1>|>tant|>  và các chi tiết;  
-g.
-s  và thông tin, địa điểm;  
-thand  tance,  tance.
-
-
